# Write FE Keywords to File

In [1]:
import pandas as pd
import json
import sys
sys.path.insert(1, "../")
sys.path.insert(1, "../utilities")

from helpers import load_posts, load_toks, load_pos

convert_to_date = lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M:%S")

from settings import TFES_FP as DB_FP, TFES_TOK_FP as TOK_FP

In [2]:
def get_align_toks(tok_fp, posts):
    # Get the corresponding tokens
    toks = {x[0]: x[1] for x in load_toks(tok_fp)}
    toks = pd.Series(toks)
    toks = toks[toks.index.isin(posts.index)]

    # Remove the posts that don't have tokens
    posts = posts[posts.index.isin(toks.index)]
    # Align the ordering of forum posts and toks
    toks = toks.loc[posts.index]

    return toks, posts

In [3]:
%%time
posts = load_posts(DB_FP)
toks, posts = get_align_toks(TOK_FP, posts)

Wall time: 16.2 s


In [4]:
from helpers import flat_earth_boards, off_topic_boards as other_boards

fe_posts = posts.query("board_id in @flat_earth_boards")
ot_posts = posts.query("board_id in @other_boards")

fe_toks = toks.loc[fe_posts.index]
ot_toks = toks.loc[ot_posts.index]

In [5]:
%%time
from language_change_methods.features import get_ngram_lr_and_ll

fe_kw = get_ngram_lr_and_ll(fe_toks, ot_toks, 1, "_")
fe_kw = fe_kw.query("LL > 3.84 and LR > 1 and freq1 > 100")

Wall time: 10 s


In [6]:
fe_kw.sort_values("LR", ascending=False).to_csv("../data/all-fe-keywords.csv")

In [7]:
top_100_kw = fe_kw.sort_values("LR", ascending=False).head(100)
top_100_kw.to_csv("../data/top-100-fe-keywords.csv")